# Ensembles

## Motivación:

Machine Learning "No free-lunch theorem":

- Distintos algoritmos funcionan correctamente en distintas situaciones

- A priori no es factible saber qué algoritmo funcionará mejor

- Por más que ajuste muy bien en el conjunto de test, aún puede estar fallando en algunos casos y podría existir otro algoritmo que a esos los ajuste mejor

A la hora de predecir, combinar distintas opiniones puede ser una excelente idea (como cuando vamos al médico) !

# Ejemplo

![](files/images/Ensembles_example.PNG)

# Formas de caracterización

### Utilizando distintos subconjuntos de variables de entrada

![](files/images/Ensembles_type_1.PNG)

### Utilizando el mismo conjunto de variables de entrada, pero distintos algoritmos

![](files/images/Ensembles_type_2.PNG)

### Dependiendo de la forma de los clasificadores base

- Homogéneos: conjunto de modelos de un mismo tipo (Ejemplo: todas redes neuronales).
- Heterogéneos: conjunto de modelos de distintos tipos (Ejemplo: redes neuronales, k-NN, logistic regression, etc.).

### Dependiendo la estructura final del ensemble (fase de predicción)

- Paralelos: Todos los modelos base realizan una predicción y se combinan de alguna manera para obtener una única salida.
- En serie: Se consultan los modelos base de forma secuencial, donde cada uno recibe los datos de entrada y la salida del modelo previo.
- Jerárquicos: Se establece una jerarquía y las salidas de los modelos base constituyen las entradas de un metamodelo padre.

# Resumen parcial

- Tenemos modelos base que sirven para obtener distintas opiniones sobre cada caso.
- Cada uno de ellos se ajusta mejor a una serie de casos y peor en otros.
- Lo ideal es combinar modelos sencillos.
- __DEBE EXISTIR VARIABILIDAD EN LAS COMBINACIONES !__

# Métodos básicos

# Fusión de etiquetas

In [1]:
# 5 clasificadores, 3 etiquetas posibles (0, 1 o 2).
outputs = [[0,1,0], [1,0,0], [1,0,0], [0,1,0], [0,1,0]]

## Voto por mayoría:

In [2]:
import numpy as np
print('Resultados parciales:', np.sum(outputs, axis=0))
print('Etiqueta final:', np.sum(outputs, axis=0).argmax(axis=0))

Resultados parciales: [2 3 0]
Etiqueta final: 1


## Voto por mayoría ponderado:

In [5]:
classifiers_weigth = [2,1,3,1,3]
partial_sum = [cw * np.array(o) for cw, o in zip(classifiers_weigth, outputs)]
print('Etiquetas ponderadas:', partial_sum)
print('Resultados parciales:', np.sum(partial_sum, axis=0))
print('Etiqueta final:', np.sum(partial_sum, axis=0).argmax(axis=0))

Etiquetas ponderadas: [array([0, 2, 0]), array([1, 0, 0]), array([3, 0, 0]), array([0, 1, 0]), array([0, 3, 0])]
Resultados parciales: [4 6 0]
Etiqueta final: 1


# Usando probabilidades

In [5]:
# 5 estimadores, 3 probabilidades para las etiquetas posibles (0, 1 o 2).
outputs = [[0.1,0.5,0.4], [0,0,1.0], [0.4,0.3,0.3], [0.2,0.7,0.1], [0.1,0.8,0.1]]

## Media aritmética

In [6]:
for l in range(3):
    print('P_{} = {}'.format(l, round(np.mean([p[l] for p in outputs]), 3)))

P_0 = 0.16
P_1 = 0.46
P_2 = 0.38


## Máximo

In [7]:
# Valor máximo para cada clase
print(np.max(outputs, axis=0))

[0.4 0.8 1. ]


# Métodos avanzados

## Bagging: Bootstrap AGGregatING

- Es una metodología o forma de construir el ensemble (no un algoritmo en sí).
- Se crean L conjuntos de datos a partir del conjunto inicial, para entrenar L modelos.
- Para predecir se combinan las salidas de los L modelos utilizando alguna técnica (generalmente voto por mayoría).
- Cada conjunto de datos se genera utilizando la técnica "boostrap" (muestreo con reemplazamiento): algunas instancias van a quedar repetidas y otras se eliminarán.
- En clasificación tiene sentido con modelos base que sean inestables ! (pequeños cambios en los datos producen resultados diferentes).
- Si tengo muchos datos los conjuntos probablemente sean similares y el método pierde eficacia.

![](files/images/bootrap_concept.PNG)

### Procedimiento (clasificación)

Fase de entrenamiento:

1) Para k clasificadores base, generar k conjuntos de datos utilizando la técnica **boostrap** a partir del conjunto de datos original.

2) Entrenar k clasificadores, cada uno con un conjunto de los creados en el paso previo.

Fase de predicción (por cada muestra o ejemplo):

1) Clasificar con los k clasificadores.

2) Combinar las salidas y devolver un único resultado.

# Random Forest

![](files/images/ensembles.png)

- Es un algoritmo que utiliza Bagging
- Ensemble homogéneo y paralelo: árboles como algoritmos base y creados de forma independiente
- Introduce aleatoriedad extra: cada nodo del árbol se genera seleccionado un subconjunto aleatorio dentro de los atributos disponibles en cada momento

Algunos hyper-parámetros para definir:

- Cantidad de árboles
- Porcentaje de variables a utilizar en cada split
- Y todos los que teníamos para un árbol (profundidad máxima, cantidad de instancias mínimas en cada nodo hoja, etc.)

### Algunas variantes ...

- Pasting: cuando los conjuntos de datos se generan sin reemplazamiento y una cantidad de datos menor a la original.
- Random Subspaces: cuando los conjuntos de datos se generan sobre un subconjunto de variables de entrada.
- Random Patches: cuando los conjuntos de datos se generan con las dos condiciones previas.

# Boosting

- Al igual que Bagging, es una metodología o forma de construir el ensemble (no un algoritmo en sí).
- Los modelos base se construyen uno después del otro de forma incremental.
- La idea subyacente es que cada modelo se concentre más en las instancias o casos donde el anterior falló.
- Se arranca entrenando un estimador base con una muestra boostrap, pero a partir de la segunda iteración la probabilidad de seleccionar a cada instancia está condicionada al error que obtuvo el modelo previo (mayor error, mayor probabilidad de ser seleccionado).

## AdaBoost (ADApting BOOSTing)

- Es un algoritmo que utiliza Boosting.
- Ensemble homogéneo y en paralelo: no está condicionado a un tipo en particular, pero el ensemble se compone de estimadores de un mismo tipo, creados de a uno.
- Los estimadores base deben ser lo más simple posibles (no tiene sentido poner uno muy bueno de entrada). *The Strength of Weak Learnability*. **Schapire, R.** (1990)

### Procedimiento

Fase de entrenamiento:

In [1]:
def AdaBoost_generation(train_data, n_iters):
    n = train_data.shape[0]
    W = [1 / n] * n
    
    models = []

    for k in range(n_iters):
        # Generar muestra utilizando W como distribución de los datos
        S = get_data_sample(train_data, W)
        # Obtener modelo y obtener predicciones
        Cl = train_model(S)
        Cl_pred = Cl.pred(S)
        # Calcular el error
        Cl_e = sum([W[i] for i, c in enumerate(Cl_pred) if c != S[i].label])
        # Verificamos que el error no sea extremo (ni muy grande ni tampoco cero).
        if Cl_e > 0 and Cl_e < 0.5:
            # Obtener factor de corrección de pesos normalizado
            w_factor = Cl_e / (1 - Cl_e)

            # Se disminuye el peso de la instancia si se clasificó correctamente
            W = [(w if Cl_pred[i] != S[i].label else w * w_factor) for i, w in enumerate(W)]
            W = W / np.max(W)
            
            models.append((Cl, w_factor))
            
    return models

In [19]:
# iteración 1
n = 10
W = [0.1] * n # [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  
Cl_e = 0.3 # el modelo falló al predecir las 3 primeras instancias
w_factor = Cl_e / (1 - Cl_e) # = 0.4285
W = [0.1, 0.1, 0.1, 0.04285, 0.04285, 0.04285, 0.04285, 0.04285, 0.04285, 0.04285]
W = W / np.sum(W)
W

array([0.16668056, 0.16668056, 0.16668056, 0.07142262, 0.07142262,
       0.07142262, 0.07142262, 0.07142262, 0.07142262, 0.07142262])

In [ ]:
Fase de predicción:

In [35]:
def AdaBoost_prediction(sample, models):
    # Inicializa un vector de pesos para las clases existentes
    c_weights = [0] * len(set(test_data.label))

    for (m, w_factor) in models:
        # Obtiene como predicción la clase
        pred_class = m.predict(sample)
        # Suma el factor de corrección del modelo al vector de pesos
        # de la clase. > error, < peso
        c_weights[pred_class] += np.log(1/w_factor)
    
    return np.argmax(c_weights)

## Gradient Boosting

- Es otra variante que utiliza Boosting.
- Al igual que los anteriores también es homogéneo, pero a la hora de predecir los modelos base se consultan en serie.
- Se basa en ir ajustando estimadores base utilizando las salidas del modelo previo como variable a predecir.
- Generalmente se utilizan árboles como estimadores base.
- Es uno de los modelos más utilizados (suele dar muy buenos resultados sin tener que preprocesar demasiado los datos).
- **xgboost** es una de las librerías más utilizadas: tiene interfaces para varios lenguajes, está súper completa y optimizada !.

### Ejemplo:

a) Vamos a entrenar un modelo A para que prediga cuál va a ser la temperatura de mañana en base a la temperatura de hoy, el viento y la humedad. Y = temperatura

b) Cuando conocemos los errores del modelo A a partir de una muestra, ahora entrenamos otro modelo B con los mismos datos de entrada pero como variable a predecir usamos el error del modelo A. Y = temperatura - prediccion_A

Una vez que entrenamos a ambos, la salida del método sería predecir con ambos modelos y sumar los resultados.

El algoritmo termina cuando el error es 0 o cae por debajo de un umbral definido, o se cumple un número máximo de iteraciones.

### Intuición

![](files/images/gradient_boosting_1.webp)

Extraído de: https://towardsdatascience.com/all-you-need-to-know-about-gradient-boosting-algorithm-part-1-regression-2520a34a502

![](files/images/gradient_boosting_2.webp)

![](files/images/gradient_boosting_3.webp)

![](files/images/gradient_boosting_4.webp)

# Bagging vs. Boosting

## Similitudes:

- Combinan estimadores base de un mismo tipo.
- Buscamos perder un poco de interpretabilidad en pos de obtener mejores resultados.
- Todos los estimadores base se generan a partir del mismo conjunto de entrenamiento.

## Diferencias:
- Con Bagging se puede paralelizar la construcción del ensemble, mientras que Boosting no.
- Bagging aisla cada modelo construido, mientras que en Boosting el conocimiento se "comparte".
- Boosting otorga pesos a los estimadores base mientras que en Bagging todos cuentan por igual.

## Resumen respecto a la fase de generación de los modelos

![](files/images/ensembles_comp.PNG)
<br>
<div style="text-align: right">ref: https://quantdare.com/</div>